In [1]:
import battery_parser as bp
import os
import pandas as pd
import numpy as np
import re
import copy
import itertools

In [ ]:
#get all files from directory
# root_dir  = r'D:\!Science\Физтех\Циклирования\28.05.24 - результаты'
# files = bp.list_files(directory=root_dir, filetype='xlsx')
# 
# #parse info
# patterns = [r'([A-Z]{3})', r'([^\\]*)', r'([0-9][^\\]*)', r'.?([0-9]?[0-9])', r'.*-(13[0-9])-.-([0-9]).*-([0-9]{14})']
# column_names = ['Chem','Tech', 'Step', 'Pouch','Potentiostat', 'Channel', 'Datetime_id']
# pattern = r'\\'.join(patterns)
# parser = bp.importing.Regex_parse()
# result = parser(strings=files,pattern=pattern,column_names= column_names)
# 
# #data enhancement
# type_convert = {i:str for i in result.columns}
# result = result.astype({'Pouch':int}, copy=True)
# result = result.astype(type_convert, copy=True)
# 
# #Create ID - unique for any pouch
# result['ID'] = result['Chem'] + result['Pouch']
# result.insert(0, 'ID', result.pop('ID'))
# 
# #Add info about thermos and cycling protocol
# thermos = {'1':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
#            '2':['LFP2', 'LFP3', 'NMC6', 'NMC7', 'LTO11', 'LTO12'],
#            '3':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7']}
# protocol = {'gost45':['NMC6', 'NMC7', 'LTO11', 'LTO12', 'LFP2', 'LFP3'],
#             'gost60':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7'],
#             'Imax45':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
#             'CCroom':['NMC10', 'NMC11', 'LTO4', 'LTO5', 'LFP1', 'LFP8']}
# 
# if bp.modifications.check_dict_intersection(thermos) or bp.modifications.check_dict_intersection(protocol):
#     print('Dict Intersection!')
# 
# thermos_creator = bp.modifications.Find_key(thermos)
# technique_creator = bp.modifications.Find_key(protocol)
# 
# result['Protocol'] = result['ID'].apply(technique_creator)
# result.insert(2, 'Protocol', result.pop('Protocol'))
# result['Thermos'] = result['ID'].apply(thermos_creator)
# result.insert(6, 'Thermos', result.pop('Thermos'))
# 
# #Create Step_id - unique value for all steps
# result.sort_values('Step', inplace=True)
# 
# step_id = [bp.modifications.step_id_creator(pouch) for name, pouch in result.groupby('ID')]
# result['Step_id'] = pd.concat(step_id, )
# result.insert(4, 'Step_id', result.pop('Step_id'))
# 
# 
# result

In [ ]:
# #Тест по размеру каждого шага
# sizes ={}
# for name, dataframe in result.groupby(['ID','Step_id']):
#     sizeof = np.sum([os.path.getsize(i) for i in dataframe['Path']])
#     sizes[name] = sizeof
# pd.Series(sizes)

In [3]:
result_path = r'D:\!Science\Физтех\Циклирования\Результаты\File_statistics.csv'
# bp.exporting.save_experiment(result, result_path)
result = bp.exporting.load_experiment(result_path)
result.sort_values(['Datetime_id', 'Path'], inplace=True)

In [4]:
def create_statistics(result_slice:pd.DataFrame, statistic_pattern):
    data = [bp.import_xls(filename, data_name_pattern='record', temp_name_pattern='auxTemp', temp_column_pattern='T1') for filename in result_slice['Path']]
    dataset = pd.concat(data,ignore_index=True)
    bp.rename_columns(dataset, {'Power(W)':'P'})
    bp.parse_time(dataset, time_column='Time', datetime_column='Datetime')
    bp.parse_time(dataset, time_column='Total Time')
    x = dataset['Total Time'].diff()
    cracks = x[x < 0]
    for idx, value in cracks.items():
        break_time = dataset['Datetime'][idx]-dataset['Datetime'][idx-1]
        dataset['Total Time'].loc[idx:] += break_time.total_seconds() + dataset['Total Time'][idx-1]


    statistics = bp.generate_statistics(dataset, group_marker=[(dataset['Time'] < dataset['Time'].shift()).cumsum(), (dataset['Step Type'] != dataset['Step Type'].shift()).cumsum()], statistics_pattern=statistic_pattern)
    statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
    statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
    bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
    statistics['Step Type'] = statistics['Step Type'].str.lower()

    return statistics, dataset

def check_time_consequence(df, column='Total Time_min'):
    if np.where(statistics[column] < statistics[column].shift(), )[0].size >0:
        raise ValueError('Not consistent data!')

In [5]:
slices = [i for i in result.groupby(['ID', 'Step_id'])]


In [6]:
statistic_pattern = {'Step Type':'unique_values',
                     'I':['mean', 'std'],
                     'Time':['range', 'diff'],
                     'Q':['max'],
                     'E':['first', 'last','mean'],
                     'T':['mean', 'min', 'max'],
                     'Datetime':['min', 'max'],
                     'Total Time':'min',
                     'Energy':'max',
                     'P':['mean', 'last']}

In [6]:
rewrite =False
for slice in slices:
    pouch, step = slice[0]
    print(pouch, step)
    
    save_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Статистика\{step}\{pouch}.csv'
    save_data_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Данные\{step}\{pouch}.csv'
    if (os.path.exists(save_path) and os.path.exists(save_data_path)) and not rewrite:
        continue
    statistics, data = create_statistics(slice[1], statistic_pattern)
    check_time_consequence(statistics,)
    bp.exporting.save_experiment(statistics, save_path)
    bp.exporting.save_experiment(data, save_data_path)

LFP1 1
LFP1 2
LFP1 3
LFP1 4
LFP1 5
LFP1 6
LFP2 1
LFP2 2
LFP2 3
LFP2 4
LFP2 5
LFP2 6
LFP3 1
LFP3 2
LFP3 3
LFP3 4
LFP3 5
LFP3 6
LFP4 1
LFP4 2
LFP4 3
LFP4 4
LFP4 5
LFP4 6
LFP5 1
LFP5 2
LFP5 3
LFP5 4
LFP5 5
LFP5 6
LFP6 1
LFP6 2
LFP6 3
LFP6 4
LFP6 5
LFP7 1
LFP7 2
LFP7 3
LFP7 4
LFP7 5
LFP8 1
LFP8 2
LFP8 3
LFP8 4
LFP8 5
LFP8 6
LTO11 1
LTO11 2
LTO11 3
LTO11 4
LTO11 5


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


LTO11 6


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


LTO12 1
LTO12 2
LTO12 3
LTO12 4
LTO12 5


...

D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NMC7 1
NMC7 2
NMC7 3
NMC7 4
NMC7 5


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NMC7 6


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
# where = np.where(statistics['Total Time_min']< statistics['Total Time_min'].shift())[0][0]
# statistics.iloc[where-10:where+10]

,Step Type,I_mean,I_std,Time_range,Time_diff,Q_max,E_first,E_last,E_mean,T_mean,T_min,T_max,Datetime_min,Datetime_max,Total Time_min,Energy_max,P_mean,P_last,Net Q,Net Energy
43021,cp dchg,-21.537354,0.017681,12.0,1.000000,0.0718,3.2569,3.2440,3.246869,59.9,59.9,59.9,2024-06-24 10:19:41,2024-06-24 10:19:52,2416608.0,0.2331,-69.928877,-69.9017,-0.0718,-0.2331
43022,cp chg,10.690867,0.037480,8.0,1.000000,0.0237,3.2440,3.2741,3.268322,59.9,59.9,59.9,2024-06-24 10:19:53,2024-06-24 10:20:00,2416620.0,0.0776,34.940922,34.9325,0.0237,0.0776
43023,rest,0.000000,0.000000,16.0,1.000000,0.0000,3.2673,3.2682,3.267412,59.9,59.9,59.9,2024-06-24 10:20:01,2024-06-24 10:20:16,2416628.0,0.0000,0.000000,0.0000,0.0000,0.0000
43024,cp dchg,-10.719164,0.006042,24.0,1.000000,0.0715,3.2682,3.2557,3.257884,59.884,59.7,60.1,2024-06-24 10:20:17,2024-06-24 10:20:40,2416644.0,0.2328,-34.921780,-34.9236,-0.0715,-0.2328
43025,cp dchg,-21.547485,0.016186,12.0,1.000000,0.0718,3.2557,3.2428,3.245700,59.853846,59.7,59.9,2024-06-24 10:20:41,2024-06-24 10:20:52,2416668.0,0.2331,-69.936654,-69.9116,-0.0718,-0.2331
43026,cp chg,10.696756,0.036942,8.0,1.000000,0.0237,3.2428,3.2732,3.267256,59.877778,59.7,59.9,2024-06-24 10:20:53,2024-06-24 10:21:00,2416680.0,0.0776,34.948722,34.9276,0.0237,0.0776
43027,rest,0.000000,0.000000,16.0,1.000000,0.0000,3.2661,3.2672,3.266394,59.888235,59.7,59.9,2024-06-24 10:21:01,2024-06-24 10:21:16,2416688.0,0.0000,0.000000,0.0000,0.0000,0.0000
43028,cp dchg,-10.724441,0.005286,36.0,1.000000,0.1072,3.2672,3.2536,3.255935,59.889189,59.7,59.9,2024-06-24 10:21:17,2024-06-24 10:21:52,2416704.0,0.3492,-34.918059,-34.9065,-0.1072,-0.3492
43029,cp dchg,-87.698289,0.638202,8.0,1.000000,0.1952,3.2536,3.1771,3.192878,59.877778,59.7,59.9,2024-06-24 10:21:53,2024-06-24 10:22:00,2416740.0,0.6222,-279.996544,-279.9960,-0.1952,-0.6222
43030,cp dchg,-54.736708,0.083290,24.0,1.000000,0.3648,3.1770,3.1933,3.195992,59.9,59.9,59.9,2024-06-24 10:22:01,2024-06-24 10:22:24,2416748.0,1.1662,-174.937728,-174.9252,-0.3648,-1.1662


In [ ]:
slice = bp.exporting.load_experiment(save_path, parse_dates = ['Datetime_min', 'Datetime_max'])
slice['Step Type'].str.findall('.*dchg')

In [137]:
slice_1 = result[result['Step_id']==3][result['ID']=='NMC10']
slice_1

C:\Users\sk804\AppData\Local\Temp\ipykernel_12556\667288033.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  slice_1 = result[result['Step_id']==3][result['ID']=='NMC10']


,ID,Chem,Protocol,Tech,Step_id,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
Index,,,,,,,,,,,,
67,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",3,3. Пункт 5 Циклирование 28 суток,10,[],132,1,20240404103731,"D:\!Science\Физтех\Циклирования\28.05.24 - результаты\NMC\Заявка № 60 Образцы C-NMC № 010, 011\3. Пункт 5 Циклирование 28 суток\010\до сбоя\BTS82-132-1-1-113-20240404103731-Циклирование 28 суток.xlsx"
68,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",3,3. Пункт 5 Циклирование 28 суток,10,[],132,1,20240411142851,"D:\!Science\Физтех\Циклирования\28.05.24 - результаты\NMC\Заявка № 60 Образцы C-NMC № 010, 011\3. Пункт 5 Циклирование 28 суток\010\после сбоя\BTS82-132-1-1-116-20240411142851-п5.xlsx"


In [140]:
# Проверка что всё время в статистике идёт подряд и нет скачков.
slices =[i for i in  result.groupby(['ID', 'Step_id'])]
slice = slices[10]
# pouch, step = slice_1[0]
# print(pouch, step)
statistics, data = create_statistics(slice_1, statistic_pattern)

LFP3 3


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [111]:
# dataset = pd.concat(data,ignore_index=True)
# bp.rename_columns(dataset, {'Power(W)':'P'})
# bp.parse_time(dataset, time_column='Time', datetime_column='Datetime')
# bp.parse_time(dataset, time_column='Total Time')
# statistics = bp.generate_statistics(dataset, group_marker=(dataset['Time'] < dataset['Time'].shift()).cumsum(), statistics_pattern=statistic_pattern)
# statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
# statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
# bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
# statistics['Step Type'] = statistics['Step Type'].str.lower()


In [7]:
data_path = r'D:\!Science\Физтех\Циклирования\Результаты\Данные'
slices = [i for i in result.groupby(['ID', 'Step_id'])]
data = {}
for slice in slices:
    pouch, step = slice[0]
    
    if step in [5,6]:
        print(pouch, step)
        filepath = fr'{data_path}\{step}\{pouch}.csv'
        data[slice[0]] = bp.exporting.load_experiment(filepath, parse_dates = ['Datetime'])

LFP1 5
LFP1 6
LFP2 5
LFP2 6
LFP3 5
LFP3 6
LFP4 5
LFP4 6
LFP5 5
LFP5 6
LFP6 5
LFP7 5
LFP8 5
LFP8 6
LTO11 5
LTO11 6
LTO12 5
LTO12 6
LTO13 5
LTO3 5
LTO4 5
LTO4 6
LTO5 5
LTO5 6
LTO6 5
LTO6 6
LTO7 5
LTO7 6
NMC10 5
NMC10 6
NMC11 5
NMC11 6
NMC12 5
NMC13 5
NMC14 5
NMC14 6
NMC15 5
NMC15 6
NMC6 5
NMC6 6
NMC7 5
NMC7 6


In [8]:
for key, dataset in data.items():
    pouch, step = key
    statistics = bp.generate_statistics(dataset, group_marker=[(dataset['Time'] < dataset['Time'].shift()).cumsum(), (dataset['Step Type'] != dataset['Step Type'].shift()).cumsum()], statistics_pattern=statistic_pattern)
    statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
    statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
    bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
    statistics['Step Type'] = statistics['Step Type'].str.lower()
    statistics.reset_index(drop=True, inplace=True)
    check_time_consequence(statistics,)
    save_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Статистика\{step}\{pouch}.csv'
    bp.exporting.save_experiment(statistics, save_path)